<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

***Задание 1***

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd
import re
import numpy as np

In [5]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head(1)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex


In [19]:
def source_reconition(traffic, region):
        """
        Функция возвращает категорию источника трафика

        """
        if traffic == 'yandex' or traffic == 'google':
            return 'organic'
        elif traffic == 'paid' or traffic == 'email': 
            if region == 'Russia':
                return 'ad'
            else:
                return 'other'
        return traffic

# создаем новый столбец class в который вносим результат работы функции source_reconition по столбцам traffic_source и region
log['source_type'] = log.apply(lambda x: source_reconition(x['traffic_source'], x['region']), axis=1)

log.tail(10)


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
18928,1550094255,c7e4bce62a,https://host.ru/8880fd00cf23066541,Australia,e721d23ec6,yandex,organic
18929,1550094260,39aeec73f8,https://host.ru/ee7c4e0674a284ee3f,Russia,9c832cf9d4,direct,direct
18930,1550094269,f1b4590410,https://host.ru/a5dda93e70318570c0,Russia,7f59a1e6a7,direct,direct
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,direct
18932,1550094280,5001544772,https://host.ru/9f90e5c423a5f976d4,Russia,072148e3ba,google,organic
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email,ad


***Задание 2***

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. 

Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [172]:
urls = pd.read_csv('URLs.txt') # читаем файл в urls 
urls_filtered = urls[urls.url.str.contains(r'\/\d{8}')] # создаем фильтрованный списокс применением регулярного выражения
urls_filtered.tail()

,url
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...
92,/world/36103095-dominikanskih-zhurnalistov-ubi...


***Задание 3***

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок.

Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [52]:
rating = pd.read_csv('ratings.csv', skiprows=0)
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
rating = pd.read_csv('ratings.csv', skiprows=0) #читаем файл в rating
# rating_100 = rating['userId'].value_counts().loc[lambda x : x > 100].to_frame().reset_index() промежуточный вариант 
# получения списка userId с количеством оценок больше 100 

"""
создаем новый Data frame, группируя по userId и timestamp max - min, затем фильтр по rating >100, затем сбрасываем индекс
для получения более ровной таблицы убираем изтаблицы столбец "rating", поскольку он нам не нужен 
и стал мало репрезентативным переименовываем "timestamp"в "lifecycle"

"""
ratings_lifecycle = rating.groupby('userId').agg({'rating':'count','timestamp':lambda x: x.max() - x.min()}) \
.query('rating>100') \
.reset_index() \
.drop('rating', 1) \
.rename(columns={'timestamp': 'lifecycle'})
ratings_lifecycle.head(10)

,userId,lifecycle
0,1,739163
1,4,62496114
2,6,3806
3,7,69546315
4,10,317722
5,15,211153396
6,17,21247022
7,18,79737120
8,19,11775
9,20,111360


***Задание 4***

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента.

*Обратите внимание, что в процессе объединения таблиц данные не должны теряться.*

In [171]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


***решение задачи "в лоб"***

поочередный merge имеющихся data frame с последующим далением столбца address

In [169]:
joined = client_base.merge(rzd, how='outer', on ='client_id')
joined.head(10)
joined_auto = joined.merge(auto, how='outer', on ='client_id')
joined_auto.head(10)
joined_all = joined_auto.merge(air, how='outer', on ='client_id')
joined_all.head(10)
joined_no_adress = joined_all.drop('address', 1)
joined_no_adress.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


***Решение с помощью метода конкатенации.***

Позволяет сократить код, но требует: 
- заведение списка из DF. 
- указание индекса в списке при конкатенации вновый DF

In [168]:
df_list = [client_base, rzd, air, auto] # Создаем список из data frame
all_data = pd.concat([i.set_index('client_id') for i in df_list], axis=1).reset_index()# конкатенация с указанием на индексы в списке
all_data.head(10) # выводим первый список со всеми данными, включая адреса
all_data_no_address = all_data.drop('address', 1)  # получаем второй список, удаляя колонку с адресами
all_data_no_address.head(10) # выводим второй список без адресов

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0
5,116,NaN,4.0,4834.0
6,117,NaN,13.0,98.0
7,118,NaN,173.0,NaN
